# Linear model hyperparameter search

In [6]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


from startup import setup_environment
from src.modeling.training import train_and_log
from src.features.preprocessors import load_dataset, list_datasets
from src.modeling.evaluate import load_model


### Load datasets

In [ ]:
datasets= list_datasets()
names = ['dataset_v5_full_hourly_selected_rfecv', 'dataset_v5_full_hourly_selected_shap', 
         'dataset_v5_slim_hourly', 'dataset_v5_full_hourly']
data_ids = [
    datasets[datasets['dataset_name'] == name].sort_values('version', ascending=False).iloc[0]['run_id']
    for name in names 
    if (datasets['dataset_name'] == name).any()
]
print(data_ids)
datasets

### Configs

In [13]:
# Hyperparameter grid
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
l1_ratios = [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0]

reg_configs = {}
for a in alphas:
    for l1 in l1_ratios:
        name = f"Alpha {a}_L1 {l1}"
        reg_configs[name] = {'alpha': a, 'l1_ratio': l1}

models = {
    'Lasso': {
        'class': Lasso,
        'param_map': lambda p: {'alpha': p['alpha']},   # p contains only alpha
        'grid': [{'alpha': a} for a in alphas]          # list of config dicts
    },
    'Ridge': {
        'class': Ridge,
        'param_map': lambda p: {'alpha': p['alpha']},
        'grid': [{'alpha': a} for a in alphas]
    },
    'ElasticNet': {
        'class': ElasticNet,
        'param_map': lambda p: {'alpha': p['alpha'], 'l1_ratio': p['l1_ratio']},
        'grid': [{'alpha': a, 'l1_ratio': l1} for a in alphas for l1 in l1_ratios]
    }
}

imputer = SimpleImputer(strategy='median')

### WeightedPipeline

In [10]:
class WeightedPipeline(Pipeline):
    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            # Forward the sample weight to the step named 'model'
            return super().fit(X, y, model__sample_weight=sample_weight)
        else:
            return super().fit(X, y)

## Model loop

### Resume

In [ ]:
start_from = (data_ids[0], 'Lasso', 0.01, None)
resume = False

In [ ]:
all_run_ids = []

for data_id in data_ids:
    for model_name, model_info in models.items():
        for config_dict in model_info['grid']:

            # Extract values for naming and tags
            alpha = config_dict['alpha']
            l1_ratio = config_dict.get('l1_ratio')   # None for Lasso/Ridge

            # Resume check: compare with (data_id, model_name, alpha, l1_ratio)
            current = (data_id, model_name, alpha, l1_ratio)
            if resume:
                if current == start_from:
                    resume = False
                else:
                    continue

             # Build model parameters (only the keys the model actually uses)
            model_params = model_info['param_map'](config_dict)
            regressor = model_info['class'](**model_params)

            pipeline = WeightedPipeline([
                ('imputer', imputer),
                ('model', regressor)
            ])

            # Build run name and description
            run_name = f"{model_name}_alpha{alpha}"
            if l1_ratio is not None:
                run_name += f"_l1{l1_ratio}"

            description = f"{model_name}, alpha={alpha}"
            if l1_ratio is not None:
                description += f", l1_ratio={l1_ratio}"

            tags = {
                "model_family": "linear",
                "model": model_name,
                "alpha": alpha,
                "l1_ratio": l1_ratio,
                "dataset_id": data_id,
                "imputation": "median"
            }

            run_id = train_and_log(
                dataset_run_id=data_id,
                model=pipeline,
                model_name=f"{model_name}_reg",
                target_transform="none",
                experiment=f"linear_hourly",
                run_name=run_name,
                description=description,
                tags=tags,
                group_size=24,
                y_baseline=None,
                test_size=0.1,
                weight_half_life=730
            )
            all_run_ids.append(run_id)